Загружаем данные по API c сайта Роскомсвободы

In [1]:
import requests

req = requests.get('https://reestr.rublacklist.net/api/v2/current/csv')
data = req.content.decode()
data_lines = []
for row in data.splitlines():
         # cоздаем список списков в виде [['ip', 'page', 'link', 'gos_organ', 'postanovlenie', 'date'], [...], [...], ...]
    data_lines.append(row.rsplit(sep=',', maxsplit=5))


In [ ]:
data_lines

Формируем датафрейм

In [260]:
import pandas as pd

df = pd.DataFrame(data_lines)

In [263]:
df

,0,1,2,3,4,5
0,ip,page,link,gos_organ,postanovlenie,date
1,"['172.67.129.252', '104.21.3.7', '2606:4700:30...",daside.club,https://daside.club/threads/debetovye-karty-ot...,Северодвинский городской суд - Архангельская о...,2а-1372/2022,2022-01-27
2,"['188.114.96.2', '188.114.97.2', '2a06:98c1:31...",*.g-live.xyz,,МВД,2022-02-18-1932,2022-02-18
3,"['188.114.96.2', '188.114.97.2', '2a06:98c1:31...",omikron-r.ru,https://omikron-r.ru/zhivotnye-jady,Ковдорский районный суд - Мурманская область,2а-74/2022,2022-02-17
4,"['188.114.97.2', '188.114.96.2', '2a06:98c1:31...",ivecop.com,https://ivecop.com/catalog/Vasu-HealthCare,Росздравнадзор,2022-02-18-2761-ПЛ,2022-02-21
...,...,...,...,...,...,...
566009,"['188.114.97.2', '188.114.96.2', '2a06:98c1:31...",hidraruzxpnew4af.net,,МВД,2020-07-15-1257,2020-07-15
566010,['91.209.226.154'],hydrarusikxpnew4bfonionon.com,,МВД,2020-07-15-1257,2020-07-15
566011,"['2606:4700:3036::ac43:b0e1', '104.21.96.107',...",livetv513.me,,Минкомсвязь,1з-44497/2022,2022-02-21
566012,"['2606:4700:3037::ac43:ad36', '2606:4700:3034:...",hydraruxzmpnsnews4af.com,,МВД,2020-07-15-1257,2020-07-15


In [265]:
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

Получаем информацию о доменах заблокированных сайтов

In [266]:
def extract_domen(text):
    try:
        domen = text.split('.')[-1]
        return domen
    except:
        return text

df['page'] = df['page'].apply(extract_domen)

In [273]:
df.page.value_counts().head(20).reset_index().to_csv('domen.csv')

In [270]:
df.page.value_counts().head(20).to_csv('domen.csv')

Получаем информацию об органах, блокирующих сайты

In [259]:
df.gos_organ.value_counts()

ФНС                                                   247053
Генпрокуратура                                         49785
Мосгорсуд                                              41180
Минкомсвязь                                            35216
суд                                                    30841
                                                       ...  
Апанасенковский районный суд - Ставропольский край         1
Чернушинский районный суд Пермскрого края                  1
Порховский районный суд Псковской области                  1
Димитровградский городской суд Ульяновской области         1
Сорский районный суд - Республика Хакасия                  1
Name: gos_organ, Length: 2264, dtype: int64

In [254]:
def region_change(text):
    try:
        region = text.split('-')[-1]
        return region
    except:
        return text

df['gos_organ'] = df['gos_organ'].apply(region_change)


In [256]:
df.gos_organ.value_counts().to_csv('regions.csv')

In [229]:
df_service = df

In [230]:
import re
def join_courts(text):
    if re.search('суд[а-я]*', text.lower()):
        return "суды"
    elif re.search('.*ФССП.*', text):
        return "судебные приставы"
    elif re.search('[\d\-/]', text):
        return None
    else:
        return text

df_service['gos_organ'] = df_service['gos_organ'].apply(join_courts)

In [231]:
df_service = df_service[df_service.gos_organ != ' Тюменской области']

In [232]:
df_service['gos_organ'].value_counts()

ФНС                         247053
суды                        168575
Генпрокуратура               49785
Минкомсвязь                  35216
Роскомнадзор                 25713
МВД                          23512
Росздравнадзор                8836
Росалкогольрегулирование      5281
Роспотребнадзор               1129
ФСКН                           620
Росмолодежь                    206
судебные приставы               67
Name: gos_organ, dtype: int64

In [233]:
df_service

,ip,page,link,gos_organ,postanovlenie,date
1,"['172.67.129.252', '104.21.3.7', '2606:4700:30...",daside.club,https://daside.club/threads/debetovye-karty-ot...,суды,2а-1372/2022,2022-01-27
2,"['188.114.96.2', '188.114.97.2', '2a06:98c1:31...",*.g-live.xyz,,МВД,2022-02-18-1932,2022-02-18
3,"['188.114.96.2', '188.114.97.2', '2a06:98c1:31...",omikron-r.ru,https://omikron-r.ru/zhivotnye-jady,суды,2а-74/2022,2022-02-17
4,"['188.114.97.2', '188.114.96.2', '2a06:98c1:31...",ivecop.com,https://ivecop.com/catalog/Vasu-HealthCare,Росздравнадзор,2022-02-18-2761-ПЛ,2022-02-21
5,"['2606:4700:3031::ac43:bba1', '172.67.187.161'...",shadow-bank.xyz,https://shadow-bank.xyz/threads/kupit-poddelny...,суды,2а-279/2022,2022-02-09
...,...,...,...,...,...,...
566009,"['188.114.97.2', '188.114.96.2', '2a06:98c1:31...",hidraruzxpnew4af.net,,МВД,2020-07-15-1257,2020-07-15
566010,['91.209.226.154'],hydrarusikxpnew4bfonionon.com,,МВД,2020-07-15-1257,2020-07-15
566011,"['2606:4700:3036::ac43:b0e1', '104.21.96.107',...",livetv513.me,,Минкомсвязь,1з-44497/2022,2022-02-21
566012,"['2606:4700:3037::ac43:ad36', '2606:4700:3034:...",hydraruxzmpnsnews4af.com,,МВД,2020-07-15-1257,2020-07-15


Преобразуем дату в формат datetime

In [ ]:
from datetime import datetime

def change_date(x):
    date = datetime.strptime(x, "%Y-%m-%d")
    return date.strftime('%m-%Y')

df_service['date'] = df_service['date'].apply(change_date)


In [ ]:
import datetime

df_service['date'] = pd.to_datetime(df_service['date']).dt.to_period('M')

In [236]:
df_service = df_service[(df_service['date'] > '12-2011') & (df_service['date'] < '01-2022')]

In [237]:
df_service['date'].value_counts()

2016-06    21557
2015-04    20532
2020-07    17775
2015-03    16745
2017-08    15322
           ...  
2014-01       25
2013-02       23
2012-08       18
2013-04       10
2012-04        3
Freq: M, Name: date, Length: 120, dtype: int64

In [238]:
df_service_viz = df_service.groupby(by=["date", "gos_organ"]).count()

In [239]:
df_service_viz = df_service_viz.drop(['ip', 'link', 'postanovlenie'], axis=1)

In [240]:
df_service_viz

page
date    gos_organ            
2012-01 суды               43
2012-02 суды              485
2012-03 суды               71
2012-04 суды                3
2012-05 суды               29
...                       ...
2021-12 Роскомнадзор      914
        Росмолодежь        14
        Роспотребнадзор    29
        ФНС              3480
        суды             7098

[685 rows x 1 columns]

In [241]:
df_service_viz.to_csv('for_services_graph_m.csv')